In [16]:
from pgtools import maf_parser
from pgtools import pangenome
from pgtools.utils import intersection_len
import typing
from pgtools.gff_parser import SimpleAnnotation
import pandas as pd
import os
from copy import deepcopy

In [2]:
maf_dir = "/home/pampuch/studia/magisterka/test_data/klebsiella_subset_old/cactus_out.full.maf"
gff_dir = "/home/pampuch/studia/magisterka/test_data/klebsiella_subset_old/gff/"

In [3]:
pg_maf = maf_parser.parse_maf(maf_dir)


In [4]:
small_maf = maf_parser.MAF(list(pg_maf.seq_collections)[:70])

In [5]:
clust_trp = small_maf.get_panaroo_cluster_triplets()

graph constructed


In [6]:
clust_trp

[(336, 335, 334),
 (335, 334, 333),
 (334, 333, 332),
 (1224, 1223, 1222),
 (1223, 1222, 1221),
 (757, 758, 759),
 (758, 759, 760),
 (759, 760, 761),
 (760, 761, 762),
 (1219, 1220, 1221),
 (1220, 1221, 1222),
 (1221, 1222, 1223),
 (1222, 1223, 1224),
 (1857, 1856, 1855),
 (1856, 1855, 1854),
 (1855, 1854, 1853),
 (1940, 1939, 1938),
 (1939, 1938, 1937),
 (1938, 1937, 1936),
 (1937, 1936, 1935),
 (762, 761, 760),
 (761, 760, 759),
 (760, 759, 758),
 (759, 758, 757),
 (265, 264, 263),
 (332, 333, 334),
 (333, 334, 335),
 (334, 335, 336),
 (423, 424, 425),
 (424, 425, 426),
 (425, 426, 427),
 (1853, 1854, 1855),
 (1854, 1855, 1856),
 (1855, 1856, 1857),
 (1935, 1936, 1937),
 (1936, 1937, 1938),
 (1937, 1938, 1939),
 (1938, 1939, 1940),
 (2203, 2204, 2205),
 (1338, 1339, 1340),
 (1338, 1339, 1341),
 (1339, 1341, 1340),
 (1339, 1340, 1341),
 (668, 669, 670),
 (2550, 2551, 2552),
 (2551, 2552, 2553),
 (425, 424, 423),
 (2205, 2204, 2203),
 (2689, 2690, 2691),
 (1340, 1339, 1338),
 (1340, 13

In [7]:
small_maf.map_to_gff(gff_dir)

In [14]:
for seq_coll in small_maf.seq_collections:
    for seq in seq_coll.sequences:
        if len(seq.mapped_annotations) > 1: 
            print(seq)
            for ann in seq.mapped_annotations:
                print(ann)

5235_2_1.contig00198: 524 - 2354, -1
7267 - 8241; strand: +; IJODNBMD_03555
8487 - 8654; strand: +; IJODNBMD_03556
5235_6_6.contig00011: 23837 - 25667, -1
7262 - 8236; strand: +; LNOJILEH_00431
8482 - 8649; strand: +; LNOJILEH_00432
5235_6_12.contig00210: 524 - 2354, -1
7261 - 8235; strand: +; LIDJBJFH_03744
8481 - 8648; strand: +; LIDJBJFH_03745
5235_5_12.contig00015: 21284 - 23114, 1
21488 - 21655; strand: -; KKAGLBML_00589
21901 - 22875; strand: -; KKAGLBML_00590
5235_6_11.contig00063: 12552 - 14382, -1
7277 - 8251; strand: +; NNHDGMCP_01955
8497 - 8664; strand: +; NNHDGMCP_01956
5235_1_4.contig00422: 2918 - 4744, -1
321 - 1295; strand: +; MGBABOJF_04120
1541 - 1708; strand: +; MGBABOJF_04121
5235_5_4.contig00016: 24332 - 26162, -1
7262 - 8236; strand: +; LLOJMOCI_00631
8482 - 8649; strand: +; LLOJMOCI_00632
5235_6_11.contig00199: 3 - 2649, -1
7171 - 8169; strand: -; NNHDGMCP_03846
8328 - 9239; strand: -; NNHDGMCP_03847
5235_5_12.contig00207: 0 - 2646, 1
330 - 1241; strand: +; KKAGL

In [18]:
block = list(small_maf.seq_collections)[0]

In [19]:
len(block)

8

In [23]:
trp = clust_trp[0]
coll_dict = small_maf.get_seq_collections_dict()

In [24]:
trp_clusters = [coll_dict[id] for id in trp]

In [25]:
# annotation triplets have to be considered separatey for each genome (not contig - adj list was constructed with broader context in mind)
# try getting three annotations for each genome, if shorter sth is consructed - discard at the end

# for each genome in a triplet
    # get all sequences with this genome from a triplet
        # get sorted list (by start) with annotatons mapped to this sequences
        # HERE IS A PROBLEM - THEY CAN BE ON DIFFERENT CONTIGS, SO CANNOT SORT THEM ALL - KEEP CLUSTERS ORDER IN MIND,
        # ---> maybe consider clusters from triplets separately and join at the end?
    # for each cluster:
        # get sequences with this genome from a triplet
        # CHECK IF THERE ARE CACES WITH MORE THAN ONE CONTIG FROM ONE GENOME IN THE CLUSTER (then just consider them separately and not sort)
        # for every contig of sequences
            # get all mapped annotations and sort them by the start
            # 
    # for this genome, join segements of annotations so that all you get all combinations
    # split all possible joints into segments of three annotation ids and append ann triplets to the value of this genome key

In [28]:
trp = clust_trp[0]
print(trp)
trp_clusters = [coll_dict[id] for id in trp]
sequences_by_clust = {clust.id:clust.sequences for clust in trp_clusters}
genomes_by_clust = {clust.id : clust.get_genome_names() for clust in trp_clusters}
print(sequences_by_clust)

(336, 335, 334)
{336: {<pgtools.maf_parser.MAFseq object at 0x7f3970e33010>, <pgtools.maf_parser.MAFseq object at 0x7f3970e33a60>, <pgtools.maf_parser.MAFseq object at 0x7f3970e33ac0>, <pgtools.maf_parser.MAFseq object at 0x7f3970e33af0>, <pgtools.maf_parser.MAFseq object at 0x7f3970e33730>, <pgtools.maf_parser.MAFseq object at 0x7f3970e339d0>, <pgtools.maf_parser.MAFseq object at 0x7f3970e33be0>}, 335: {<pgtools.maf_parser.MAFseq object at 0x7f3970e41450>, <pgtools.maf_parser.MAFseq object at 0x7f3970e406d0>, <pgtools.maf_parser.MAFseq object at 0x7f3970e420e0>, <pgtools.maf_parser.MAFseq object at 0x7f3970e41750>, <pgtools.maf_parser.MAFseq object at 0x7f3970e41780>, <pgtools.maf_parser.MAFseq object at 0x7f3970e409d0>, <pgtools.maf_parser.MAFseq object at 0x7f3970e417e0>}, 334: {<pgtools.maf_parser.MAFseq object at 0x7f3970e259f0>, <pgtools.maf_parser.MAFseq object at 0x7f3970e25690>, <pgtools.maf_parser.MAFseq object at 0x7f3970e1ba90>, <pgtools.maf_parser.MAFseq object at 0x7f3970

# Crucial
1. Remember to check overlap on gff coords (but annotation ids already in gff so ok)
2. 

In [2]:
summaries_dir = "/home/pampuch/studia/magisterka/final_summaries/"
panaroo_struct_dir = os.path.join(summaries_dir, "struct_triplets")
panaroo_seqs_summaries = os.path.join(summaries_dir, "annots_summaries", "panaroo_models")
annots_csv = "panaroo_annots.csv"
seqs_csv = "panaroo_seqs.csv"
triplets_file = "struct_presence_absence.Rtab"
dataset = "klebsiella_subset"
rtab_file = os.path.join(panaroo_struct_dir, dataset, triplets_file)
seqs_file = os.path.join(panaroo_seqs_summaries, dataset, seqs_csv)

In [11]:
for dir in os.listdir(panaroo_struct_dir):
    print(dir)

sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1_fragmented
sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1_contaminated
klebsiella_subset
sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1
GPSC_subset


In [6]:
for dir in os.listdir(panaroo_seqs_summaries):
    print(dir)

sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1_fragmented
sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1_contaminated
klebsiella_subset
sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1
GPSC_subset


In [3]:
dataset = "klebsiella_subset"

In [4]:
rtab_file = os.path.join(panaroo_struct_dir, dataset, triplets_file)
seqs_file = os.path.join(panaroo_seqs_summaries, dataset, seqs_csv)

In [6]:
df_triplets = pd.read_csv(os.path.join(panaroo_struct_dir, dataset, triplets_file), sep = "\t")

In [7]:
df_triplets

,Gene,5150_1_3,5150_2_2,5150_3_5,5151_2_6,5151_6_6,5193_1_5,5193_2_6,5193_8_2,5197_2_1,...,5197_7_5,5235_1_4,5235_2_1,5235_5_12,5235_5_4,5235_6_11,5235_6_12,5235_6_6,5299_1_3,5299_7_4
0,group_1763-group_1916-tppB,0,1,0,0,0,0,1,0,0,...,1,0,0,0,0,0,1,0,0,0
1,group_1763-group_1916-group_3479,1,0,1,1,1,1,0,1,1,...,0,0,1,1,1,1,0,1,1,1
2,group_1916-tppB-gst,0,1,0,0,0,0,1,0,0,...,1,0,0,0,0,0,1,0,0,0
3,group_3479-tppB-gst,1,0,1,1,1,1,0,1,1,...,0,0,1,1,1,1,0,1,1,1
4,group_3856-group_439-group_4212,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1896,group_3328-group_6525-group_6411,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1897,group_2533-group_6526-group_6527,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1898,group_6396-group_6527-group_6526,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1899,group_5354-group_6529-group_2677,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1


In [21]:
all_clust_trp = [tuple(i.split("-")) for i in df_triplets["Gene"].to_numpy()]

In [23]:
all_clust_trp[:3]

[('group_1763', 'group_1916', 'tppB'),
 ('group_1763', 'group_1916', 'group_3479'),
 ('group_1916', 'tppB', 'gst')]

In [25]:
for i in all_clust_trp: assert len(i) == 3

In [17]:
def clust_trp_from_rtab(rtab):
    df_triplets = pd.read_csv(rtab, sep = "\t")
    return [tuple(i.split("-")) for i in df_triplets["Gene"].to_numpy()]
clust_trp_from_rtab(rtab_file)[:3]

[('group_1763', 'group_1916', 'tppB'),
 ('group_1763', 'group_1916', 'group_3479'),
 ('group_1916', 'tppB', 'gst')]

In [9]:
df_seqs = pd.read_csv(seqs_file)
df_seqs.rename(columns = {"seq name.1" : "seq start", "seq start" : "seq end", "seq end":"strand", "seq strand" : "CDS new"}, inplace=True)
df_seqs = df_seqs.drop("CDS", axis = 1)
df_seqs = df_seqs.rename(columns = {"CDS new" : "CDS"})

In [52]:
df_seqs

,cluster name,cluster size,core status,seq name,seq start,seq end,strand,CDS
0,group_7230,20,True,5150_3_5.contig00022,61604,62405,-1,JGFEPBDE_02849
1,group_7230,20,True,5150_2_2.contig00027,61370,62171,-1,ILAAFBJJ_04361
2,group_7230,20,True,5150_1_3.contig00024,61692,62493,1,AOMCCKAJ_03991
3,group_7230,20,True,5151_2_6.contig00018,61455,62256,1,NALFBGJE_02497
4,group_7230,20,True,5151_6_6.contig00027,61539,62340,-1,NJAAJKJA_03996
...,...,...,...,...,...,...,...,...
106895,abrB,20,True,5235_6_11.contig00205,6297,7350,-1,NNHDGMCP_03892
106896,abrB,20,True,5235_6_12.contig00073,6205,7258,-1,LIDJBJFH_02042
106897,abrB,20,True,5235_6_6.contig00612,110,779,1,LNOJILEH_05270
106898,abrB,20,True,5299_1_3.contig00180,120,1173,-1,OBOFFKJP_04607


In [11]:
row = list(df_seqs.iterrows())[0]

In [12]:
def row_to_seq(row):
    # print(row[1]["seq name"])
    seq = pangenome.BaseSeq(row[1]["seq name"], row[1]["seq start"], row[1]["seq end"], row[1]["strand"], -1, "maf")
    seq.annotation_ids = [row[1]["CDS"]]
    seq.cluster_id = row[1]["cluster name"]
    return seq
seq = row_to_seq(row)
print(seq)

5150_3_5.contig00022: 61604 - 62405, -1


In [13]:
all_seqs = {clust_id : set() for clust_id in df_seqs["cluster name"].unique()}
for row in df_seqs.iterrows():
    all_seqs[row[1]["cluster name"]].add(row_to_seq(row))

In [14]:
all_clusts = [pangenome.SeqCollection(k, v) for k,v in all_seqs.items()]

In [15]:
pg_model = pangenome.Pangenome(all_clusts)

In [16]:
for coll in pg_model.seq_collections:
    for seq in coll.sequences:
        print(seq)
        print(seq.annotation_ids)
        print(seq.cluster_id)
        break
    break

5193_2_6.contig00060: 8447 - 11594, 1
['FGIHKDBE_04918']
group_30


In [2]:
summaries_dir = "/home/pampuch/studia/magisterka/final_summaries/"
panaroo_struct_dir = os.path.join(summaries_dir, "struct_triplets")
panaroo_seqs_summaries = os.path.join(summaries_dir, "annots_summaries", "panaroo_models")
annots_csv = "panaroo_annots.csv"
seqs_csv = "panaroo_seqs.csv"
triplets_file = "struct_presence_absence.Rtab"
dataset = "klebsiella_subset"
rtab_file = os.path.join(panaroo_struct_dir, dataset, triplets_file)
seqs_file = os.path.join(panaroo_seqs_summaries, dataset, seqs_csv)

In [3]:
def row_to_seq(row):
    # print(row[1]["seq name"])
    seq = pangenome.BaseSeq(row[1]["seq name"], row[1]["seq start"], row[1]["seq end"], row[1]["strand"], -1, "maf")
    seq.annotation_ids = [row[1]["CDS"]]
    seq.cluster_id = row[1]["cluster name"]
    return seq
    
def csv_to_pg(seqs_csv):
    df_seqs = pd.read_csv(seqs_csv)
    df_seqs.rename(columns = {"seq name.1" : "seq start", "seq start" : "seq end", "seq end":"strand", "seq strand" : "CDS new"}, inplace=True)
    df_seqs = df_seqs.drop("CDS", axis = 1)
    df_seqs = df_seqs.rename(columns = {"CDS new" : "CDS"})
    all_seqs = {clust_id : set() for clust_id in df_seqs["cluster name"].unique()}
    for row in df_seqs.iterrows():
        all_seqs[row[1]["cluster name"]].add(row_to_seq(row))
    all_clusts = [pangenome.SeqCollection(k, v) for k,v in all_seqs.items()]
    pg_model = pangenome.Pangenome(all_clusts)
    return pg_model

def clust_trp_from_rtab(rtab):
    df_triplets = pd.read_csv(rtab, sep = "\t")
    return [tuple(i.split("-")) for i in df_triplets["Gene"].to_numpy()]
# clust_trp_from_rtab(rtab_file)[:3]

In [42]:
pg_model = csv_to_pg(seqs_file)
all_trp = clust_trp_from_rtab(rtab_file)

In [6]:
trp = all_trp[0]

In [7]:
trp

('group_1763', 'group_1916', 'tppB')

In [91]:
colls_by_id_0 = pg_model.get_seq_collections_dict()

In [92]:
colls_by_id = deepcopy(colls_by_id_0)

In [93]:
trp_colls = [colls_by_id[id] for id in trp]

In [94]:
trp_colls

In [95]:
trp_colls_dummy = deepcopy(trp_colls)

In [96]:
some_seq = deepcopy(list(trp_colls[0].sequences)[0])
print(some_seq)
some_seq.seq_name = some_seq.get_genome_name() + ".DUMMY_CONTIG_0"
some_seq.annotation_ids = ["DUMMY_ANN_1"]

5299_1_3.contig00147: 2003 - 2702, -1


In [97]:
other_seq = deepcopy(some_seq)
list(trp_colls[0].sequences)[0].annotation_ids.append("DUMMY_ANN_2")
list(trp_colls[0].sequences)[6].annotation_ids.append("DUMMY_ANN_3")

In [99]:
# modify so that you consider paralogs and multiple annot ids
trp_colls[0].sequences.add(some_seq)

In [100]:
def cluster_to_annots_by_genome(clst: pangenome.SeqCollection):
    annots_by_genomes = {g: []  for g in clst.get_genome_names()}
    # print(genomes)
    for seq in clst.sequences:
        genome = seq.get_genome_name()
        for ann_id in seq.annotation_ids:
        #in non panaroo - consider overlaps
            #if overlap is small - add [], else, add id to [-1]
            annots_by_genomes[genome].append([ann_id])
        
    return annots_by_genomes
cluster_to_annots_by_genome(trp_colls[0])

{'5151_6_6': [['NJAAJKJA_01769']],
 '5235_5_12': [['KKAGLBML_01388']],
 '5235_6_12': [['LIDJBJFH_03997']],
 '5235_6_6': [['LNOJILEH_02386']],
 '5299_7_4': [['MOOPCBGB_05194']],
 '5150_3_5': [['JGFEPBDE_00004']],
 '5193_1_5': [['AHIKKBJF_05387']],
 '5150_2_2': [['ILAAFBJJ_03029']],
 '5197_2_1': [['PDEDMGPD_03466']],
 '5197_7_5': [['EHMCBADO_02460']],
 '5197_7_4': [['IMGDPOBN_03658']],
 '5235_1_4': [['MGBABOJF_04616']],
 '5193_2_6': [['FGIHKDBE_01220']],
 '5235_5_4': [['LLOJMOCI_04272']],
 '5150_1_3': [['AOMCCKAJ_00004']],
 '5151_2_6': [['NALFBGJE_02652']],
 '5235_6_11': [['NNHDGMCP_03563']],
 '5193_8_2': [['MLKBEPLK_03618']],
 '5299_1_3': [['OBOFFKJP_04245'], ['DUMMY_ANN_2'], ['DUMMY_ANN_1']],
 '5235_2_1': [['IJODNBMD_04164'], ['DUMMY_ANN_3']]}

In [101]:
all_genomes = sorted(list(pg_model.get_genome_names()))
all_genomes

['5150_1_3',
 '5150_2_2',
 '5150_3_5',
 '5151_2_6',
 '5151_6_6',
 '5193_1_5',
 '5193_2_6',
 '5193_8_2',
 '5197_2_1',
 '5197_7_4',
 '5197_7_5',
 '5235_1_4',
 '5235_2_1',
 '5235_5_12',
 '5235_5_4',
 '5235_6_11',
 '5235_6_12',
 '5235_6_6',
 '5299_1_3',
 '5299_7_4']

In [200]:
# def clst_triplets_to_csv(clts_triplets, genomes, csv_out):
    # adnotations are cluster specific, so making rtab presence absence does not make sense

    

def clst_triplet_to_annot_triplet(clst_trp, pg_genomes, res_csv):

    annots_by_clst = []
    for clst in clst_trp:
        annots_by_clst.append(cluster_to_annots_by_genome(clst))
    # print(annots_by_clst)
    # triplets_by_genome = {G: [[],[],[]] for G in pg_genomes}
    all_genome_chains = {G:[] for G in pg_genomes}
    for genome in pg_genomes:
        genome_chains = [[]]
        # if genome in annots_by_clst[0]:
        #     genome_chains = annots_by_clst[0][genome]
        # print(genome_chains)
        # print(genome)
        for annots in annots_by_clst:
            # if genome was not in a given cluster - append False - means, that there will be a break in a chain
            # print(annots[genome])
            # print(annots[genome])
            new_chains = []
            if genome in annots:
                for ann in annots[genome]:
                    for chain in genome_chains:
                        new_chains.append(chain + ann)
                        # print(new_chains)
                genome_chains = new_chains
            else:
                all_genome_chains[genome] += genome_chains
                genome_chains = [[]]
        
        all_genome_chains[genome] += genome_chains
                
    # print(genome_chains)
    for genome, chains in all_genome_chains.items():
        # print(genome, chains)
        for chain in chains:
            for i in range(0, len(chain)-2):
                # print(chain[i:i+3])
                triplet = "-".join(chain[i:i+3])
                # print(triplet)
                res_csv.write(f"{genome},{triplet}\n")
                res_csv.flush()
                
        # print("-"*80)
# clst_triplet_to_annot_triplet(trp_colls, all_genomes)

In [201]:
csv_out = "trp.csv"
res_csv = open(csv_out, "w")
res_csv.write("genome,triplet\n")
res_csv.flush()

for trp in all_trp:
    trp_colls = [colls_by_id[id] for id in trp]
    clst_triplet_to_annot_triplet(trp_colls, all_genomes, res_csv)
    
res_csv.close()

In [11]:
# annotation triplets have to be considered separatey for each genome (not contig - adj list was constructed with broader context in mind)
# try getting three annotations for each genome, if shorter sth is consructed - discard at the end

# for each genome in a triplet
    # get all sequences with this genome from a triplet
        # get sorted list (by start) with annotatons mapped to this sequences
        # HERE IS A PROBLEM - THEY CAN BE ON DIFFERENT CONTIGS, SO CANNOT SORT THEM ALL - KEEP CLUSTERS ORDER IN MIND,
        # ---> maybe consider clusters from triplets separately and join at the end?
    # for each cluster:
        # get sequences with this genome from a triplet
        # CHECK IF THERE ARE CACES WITH MORE THAN ONE CONTIG FROM ONE GENOME IN THE CLUSTER (then just consider them separately and not sort)
        # for every contig of sequences
            # get all mapped annotations and sort them by the start
            # 
    # for this genome, join segements of annotations so that all you get all combinations
    # split all possible joints into segments of three annotation ids and append ann triplets to the value of this genome key